# (4~10)년도 예측

In [ ]:
import os
os.chdir('./../../')
from src.basic_utils import *
from src.modeling_utils import *

from sklearn.metrics.pairwise import cosine_similarity
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.models import load_model
from keras.initializers import glorot_normal, Zeros, Ones
import keras.backend as K
from keras.optimizers import RMSprop
import tensorflow as tf

In [ ]:
data_path = 'data/'

train = pd.read_parquet(data_path+'train(song_trim).parquet')
val = pd.read_json(data_path+'val(lower).json')
# test = pd.read_json(data_path+'test(lower).json')
# song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

### make_file.ipynb 내에서 만들어진 tag_song_dict / song_tag_dict 불러오기
# tag_song_dict = load_json(data_path+'tag_song_dict.json')
song_tag_dict = load_json(data_path+'song_tag_dict.json')

In [ ]:
# file_path = '/kaggle/input/yearly-pv-file-0-10-11-13-14-15/train_DF_%s.parquet' %small_years

small_years = [i for i in range(4,11)]
train_pv = pd.read_parquet('/kaggle/input/yearly-pv-file-0-10-11-13-14-15/train_DF_[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10].parquet')

In [ ]:
print(train_pv.shape)
display(train_pv.head(2))
display(train_pv.tail(2))

(21630, 5719)


id,49,59,92,104,121,129,204,208,252,290,...,153190,153218,153221,153229,153230,153258,153313,153333,153392,153426
song,,,,,,,,,,,,,,,,,,,,,
62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


id,49,59,92,104,121,129,204,208,252,290,...,153190,153218,153221,153229,153230,153258,153313,153333,153392,153426
song,,,,,,,,,,,,,,,,,,,,,
707896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
707960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## autoencoder

In [1]:
def make_autoencoder(train_pv):
    input_dim = train_pv.shape[1]
    c = 500
    model = Sequential()
    model.add(Dense(c, input_dim=input_dim, activation='relu'))
    model.add(Dense(input_dim, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse') # (optimizer='adadelta', loss='binary_crossentropy')
    
    tmp = train_pv.values
    history = model.fit(tmp, tmp,
                    epochs=4,
                    batch_size=256,
                    shuffle=True)
    
#     model.summary()
    encoder = Model(model.input, model.layers[0].output)
    encoded_imgs = encoder.predict(tmp)
    encoder.save("encoder%s" %small_years)
    del tmp, model, encoder
    gc.collect()
    
    train_data = pd.DataFrame(encoded_imgs)
    del encoded_imgs
    gc.collect()
    
    cosine_array = cosine_similarity(train_data, train_data)
    print('cosine_array complete')
    index2id = {i:u for i, u in enumerate((train_pv.index))}

    cf_dic = {}
    for i in tqdm(range(len(cosine_array))):
        song_id= list(map(index2id.get, list(np.argsort(-cosine_array[i])[1:201])))
        value= list(np.sort(cosine_array[0])[::-1][1:201])
        cf_dic[index2id[i]] = list(zip(song_id,value))

    del cosine_array, train_data
    gc.collect()
    
    return cf_dic

In [ ]:
cf_dic = make_autoencoder(train_pv)

5719


In [ ]:
no_tag, no_song, yes_index, no_both = check_target_type(val)

노래가 하나도 없음 :  1232
노래, 태그 둘 다 존재함 :  4190
태그가 없음 :  4507
둘다 없음 :  811


In [ ]:
v1v3_index= no_tag+yes_index

In [ ]:
train['year'] = train['updt_date'].apply( lambda x: int(x[2:4]) )
train['date'] = train['updt_date'].apply( lambda x: int(str(x[2:4]) + str(x[5:7])) )
val['year'] = val['updt_date'].apply( lambda x: int(x[2:4]) )
val['date'] = val['updt_date'].apply( lambda x: int(str(x[2:4]) + str(x[5:7])) )

year_list = list(train['year'].unique())
year_list= sorted(year_list)

v1v3 = val[val.index.isin(v1v3_index)]

val_tmp = v1v3[v1v3['year'].isin(small_years)]
print("val year shape", val_tmp.shape)

val year shape (484, 8)


In [ ]:
date_list = sorted(list(train['date'].unique()))

date_dict = {}
for i,k in enumerate(date_list):
    date_dict[k] = i

popular_date_tag = make_popular_date_dict('tags', train, date_list, 13)
popular_date_song = make_popular_date_dict('songs', train, date_list, 9)

In [ ]:
def f7(seq):
    '''
    list에 있는 중복 데이터는 삭제하고, 순서는 유지하는 함수
    set만 사용하면 순서가 뒤엉키게 됨
    '''
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [ ]:
v1v3_predict = []
for i in range(len(val_tmp)):
    date = date_dict[val_tmp.iloc[i]['date']]
    
    ################ song 채우기 ###############################
    songs = val_tmp.loc[i]['songs']
    songs = [x for x in songs if x in cf_dic]

    if len(songs):
        tmp_list = []
        for song in songs:
            tmp_list.extend(cf_dict[song]) 
        tmp_list.sort(key = lambda x: x[1], reverse = True)    
        tmp_list = f7([i[0] for i in tmp_list if i[1] > 0])
        cur_song = tmp_list[:200]
    else:
        cur_song = popular_date_song[date]['songs']

    cur_song = remove_seen(val_tmp.iloc[i]['songs'], cur_song)[:100]

    #중복 케이스 때문에 부족하면 popular song으로 다시 채워줌
    if len(cur_song) != 100:
        update_song = remove_seen(cur_song, popular_date_song[date]['songs'])
        cur_song.extend(update_song)
        cur_song = cur_song[:200]
    
    
    ############### tag 채우기 ##################################
    tag_counter = Counter()
    for song in val_tmp.iloc[i]['songs']:
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key= lambda x:x[1], reverse = True)        
    tag_counter = tag_counter[:100] ## 넉넉히 100개 잡는다. 
    
    cur_tag = []
    for k in tag_counter:
        cur_tag.append(k[0])     
        
    if (len(cur_tag) < 100) & (len(cur_tag) > 0 ):
        ## 부족하면 채워준다.
        update_tag = remove_seen(cur_tag, popular_date_tag[date]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:100]
    if len(cur_tag) == 0:
        cur_tag = popular_date_tag[date]['tags'][:100]
        
    #############################################################    

    v1v3_predict.append({
        "id" : val_tmp.iloc[i]['id'],
        "songs": remove_seen(val_tmp.iloc[i]['songs'], cur_song)[:100],
        "tags": remove_seen(val_tmp.iloc[i]['tags'], cur_tag)[:10],
     })    


In [ ]:
write_json(v1v3_predict, 'v1v3_%s_predict.json' %small_years)